In [1]:
import sklearn as sk
import sklearn.tree as tree
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
import pickle
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [3]:
df = pd.read_csv('SpotifyFeatures.csv')
df

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.61100,0.389,99373,0.910,0.000000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.24600,0.590,137373,0.737,0.000000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.95200,0.663,170267,0.131,0.000000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.70300,0.240,152427,0.326,0.000000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.95000,0.331,82625,0.225,0.123000,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232720,Soul,Slave,Son Of Slide,2XGLdVl7lGeq8ksM6Al7jT,39,0.00384,0.687,326240,0.714,0.544000,D,0.0845,-10.626,Major,0.0316,115.542,4/4,0.962
232721,Soul,Jr Thomas & The Volcanos,Burning Fire,1qWZdkBl4UVPj9lK6HuuFM,38,0.03290,0.785,282447,0.683,0.000880,E,0.2370,-6.944,Minor,0.0337,113.830,4/4,0.969
232722,Soul,Muddy Waters,(I'm Your) Hoochie Coochie Man,2ziWXUmQLrXTiYjCg2fZ2t,47,0.90100,0.517,166960,0.419,0.000000,D,0.0945,-8.282,Major,0.1480,84.135,4/4,0.813
232723,Soul,R.LUM.R,With My Words,6EFsue2YbIG4Qkq8Zr9Rir,44,0.26200,0.745,222442,0.704,0.000000,A,0.3330,-7.137,Major,0.1460,100.031,4/4,0.489


,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.61100,0.389,99373,0.910,0.000000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.24600,0.590,137373,0.737,0.000000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.95200,0.663,170267,0.131,0.000000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.70300,0.240,152427,0.326,0.000000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.95000,0.331,82625,0.225,0.123000,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232720,Soul,Slave,Son Of Slide,2XGLdVl7lGeq8ksM6Al7jT,39,0.00384,0.687,326240,0.714,0.544000,D,0.0845,-10.626,Major,0.0316,115.542,4/4,0.962
232721,Soul,Jr Thomas & The Volcanos,Burning Fire,1qWZdkBl4UVPj9lK6HuuFM,38,0.03290,0.785,282447,0.683,0.000880,E,0.2370,-6.944,Minor,0.0337,113.830,4/4,0.969
232722,Soul,Muddy Waters,(I'm Your) Hoochie Coochie Man,2ziWXUmQLrXTiYjCg2fZ2t,47,0.90100,0.517,166960,0.419,0.000000,D,0.0945,-8.282,Major,0.1480,84.135,4/4,0.813
232723,Soul,R.LUM.R,With My Words,6EFsue2YbIG4Qkq8Zr9Rir,44,0.26200,0.745,222442,0.704,0.000000,A,0.3330,-7.137,Major,0.1460,100.031,4/4,0.489


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232725 entries, 0 to 232724
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   genre             232725 non-null  object 
 1   artist_name       232725 non-null  object 
 2   track_name        232725 non-null  object 
 3   track_id          232725 non-null  object 
 4   popularity        232725 non-null  int64  
 5   acousticness      232725 non-null  float64
 6   danceability      232725 non-null  float64
 7   duration_ms       232725 non-null  int64  
 8   energy            232725 non-null  float64
 9   instrumentalness  232725 non-null  float64
 10  key               232725 non-null  object 
 11  liveness          232725 non-null  float64
 12  loudness          232725 non-null  float64
 13  mode              232725 non-null  object 
 14  speechiness       232725 non-null  float64
 15  tempo             232725 non-null  float64
 16  time_signature    23

In [5]:
df.shape

(232725, 18)

(232725, 18)

In [6]:
df.describe()

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
count,232725.000000,232725.000000,232725.000000,2.327250e+05,232725.000000,232725.000000,232725.000000,232725.000000,232725.000000,232725.000000,232725.000000
mean,41.127502,0.368560,0.554364,2.351223e+05,0.570958,0.148301,0.215009,-9.569885,0.120765,117.666585,0.454917
std,18.189948,0.354768,0.185608,1.189359e+05,0.263456,0.302768,0.198273,5.998204,0.185518,30.898907,0.260065
min,0.000000,0.000000,0.056900,1.538700e+04,0.000020,0.000000,0.009670,-52.457000,0.022200,30.379000,0.000000
25%,29.000000,0.037600,0.435000,1.828570e+05,0.385000,0.000000,0.097400,-11.771000,0.036700,92.959000,0.237000
50%,43.000000,0.232000,0.571000,2.204270e+05,0.605000,0.000044,0.128000,-7.762000,0.050100,115.778000,0.444000
75%,55.000000,0.722000,0.692000,2.657680e+05,0.787000,0.035800,0.264000,-5.501000,0.105000,139.054000,0.660000
max,100.000000,0.996000,0.989000,5.552917e+06,0.999000,0.999000,1.000000,3.744000,0.967000,242.903000,1.000000


,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
count,232725.000000,232725.000000,232725.000000,2.327250e+05,232725.000000,232725.000000,232725.000000,232725.000000,232725.000000,232725.000000,232725.000000
mean,41.127502,0.368560,0.554364,2.351223e+05,0.570958,0.148301,0.215009,-9.569885,0.120765,117.666585,0.454917
std,18.189948,0.354768,0.185608,1.189359e+05,0.263456,0.302768,0.198273,5.998204,0.185518,30.898907,0.260065
min,0.000000,0.000000,0.056900,1.538700e+04,0.000020,0.000000,0.009670,-52.457000,0.022200,30.379000,0.000000
25%,29.000000,0.037600,0.435000,1.828570e+05,0.385000,0.000000,0.097400,-11.771000,0.036700,92.959000,0.237000
50%,43.000000,0.232000,0.571000,2.204270e+05,0.605000,0.000044,0.128000,-7.762000,0.050100,115.778000,0.444000
75%,55.000000,0.722000,0.692000,2.657680e+05,0.787000,0.035800,0.264000,-5.501000,0.105000,139.054000,0.660000
max,100.000000,0.996000,0.989000,5.552917e+06,0.999000,0.999000,1.000000,3.744000,0.967000,242.903000,1.000000


In [7]:
count = df.nunique()
count = count.sort_values()
count

mode                     2
time_signature           5
key                     12
genre                   27
popularity             101
danceability          1295
speechiness           1641
valence               1692
liveness              1732
energy                2517
acousticness          4734
instrumentalness      5400
artist_name          14564
loudness             27923
duration_ms          70749
tempo                78512
track_name          148615
track_id            176774
dtype: int64

mode                     2
time_signature           5
key                     12
genre                   27
popularity             101
danceability          1295
speechiness           1641
valence               1692
liveness              1732
energy                2517
acousticness          4734
instrumentalness      5400
artist_name          14564
loudness             27923
duration_ms          70749
tempo                78512
track_name          148615
track_id            176774
dtype: int64

In [8]:
df.drop(['track_id', 'artist_name','track_name'],axis=1,inplace=True)

In [9]:
count = df.nunique()
count = count.sort_values()
count

mode                    2
time_signature          5
key                    12
genre                  27
popularity            101
danceability         1295
speechiness          1641
valence              1692
liveness             1732
energy               2517
acousticness         4734
instrumentalness     5400
loudness            27923
duration_ms         70749
tempo               78512
dtype: int64

mode                    2
time_signature          5
key                    12
genre                  27
popularity            101
danceability         1295
speechiness          1641
valence              1692
liveness             1732
energy               2517
acousticness         4734
instrumentalness     5400
loudness            27923
duration_ms         70749
tempo               78512
dtype: int64

In [10]:
df.skew()

popularity         -0.336390
acousticness        0.534242
danceability       -0.378278
duration_ms         9.893376
energy             -0.400223
instrumentalness    1.819766
liveness            2.074093
loudness           -1.662208
speechiness         3.311269
tempo               0.403350
valence             0.144131
dtype: float64

popularity         -0.336390
acousticness        0.534242
danceability       -0.378278
duration_ms         9.893376
energy             -0.400223
instrumentalness    1.819766
liveness            2.074093
loudness           -1.662208
speechiness         3.311269
tempo               0.403350
valence             0.144131
dtype: float64

In [11]:
columns = ['speechiness', 'acousticness', 'instrumentalness', 'liveness']
for col in columns:
    df[col] = np.sqrt(df[col])

In [12]:
df.skew()

popularity         -0.336390
acousticness        0.054789
danceability       -0.378278
duration_ms         9.893376
energy             -0.400223
instrumentalness    1.514937
liveness            1.399108
loudness           -1.662208
speechiness         2.292474
tempo               0.403350
valence             0.144131
dtype: float64

popularity         -0.336390
acousticness        0.054789
danceability       -0.378278
duration_ms         9.893376
energy             -0.400223
instrumentalness    1.514937
liveness            1.399108
loudness           -1.662208
speechiness         2.292474
tempo               0.403350
valence             0.144131
dtype: float64

In [13]:
columns = ['speechiness', 'instrumentalness', 'liveness']
for col in columns:
    df[col] = np.sqrt(df[col])

In [14]:
df.skew()

popularity         -0.336390
acousticness        0.054789
danceability       -0.378278
duration_ms         9.893376
energy             -0.400223
instrumentalness    1.123633
liveness            1.043329
loudness           -1.662208
speechiness         1.769583
tempo               0.403350
valence             0.144131
dtype: float64

popularity         -0.336390
acousticness        0.054789
danceability       -0.378278
duration_ms         9.893376
energy             -0.400223
instrumentalness    1.123633
liveness            1.043329
loudness           -1.662208
speechiness         1.769583
tempo               0.403350
valence             0.144131
dtype: float64

In [15]:
columns = ['speechiness', 'instrumentalness']
for col in columns:
    df[col] = np.sqrt(df[col])

In [16]:
df.skew()

popularity         -0.336390
acousticness        0.054789
danceability       -0.378278
duration_ms         9.893376
energy             -0.400223
instrumentalness    0.531948
liveness            1.043329
loudness           -1.662208
speechiness         1.527922
tempo               0.403350
valence             0.144131
dtype: float64

popularity         -0.336390
acousticness        0.054789
danceability       -0.378278
duration_ms         9.893376
energy             -0.400223
instrumentalness    0.531948
liveness            1.043329
loudness           -1.662208
speechiness         1.527922
tempo               0.403350
valence             0.144131
dtype: float64

In [17]:

genre = (df["genre"].str.strip()).str.lower()


In [18]:
def genre_splitter(genre):
    result = genre.copy()
    result = result.str.split(" ",1)
    for i in range(len(result)):
        if (len(result[i]) > 1):
            result[i] = [result[i][1]]
    return result.str.join('')

In [19]:
genre_m1 = genre.copy()
while(max((genre_m1.str.split(" ", 1)).str.len()) > 1):
    genre_m1 = genre_splitter(genre_m1)

In [20]:
len(genre_m1.unique())

26

26

In [21]:
genre_m1.value_counts()

music          14756
comedy          9681
soundtrack      9646
indie           9543
jazz            9441
pop             9386
electronic      9377
folk            9299
hip-hop         9295
rock            9272
alternative     9263
classical       9256
rap             9232
world           9096
soul            9089
blues           9023
r&b             8992
anime           8936
reggaeton       8927
ska             8874
reggae          8771
dance           8701
country         8664
opera           8280
movie           7806
capella          119
Name: genre, dtype: int64

music          14756
comedy          9681
soundtrack      9646
indie           9543
jazz            9441
pop             9386
electronic      9377
folk            9299
hip-hop         9295
rock            9272
alternative     9263
classical       9256
rap             9232
world           9096
soul            9089
blues           9023
r&b             8992
anime           8936
reggaeton       8927
ska             8874
reggae          8771
dance           8701
country         8664
opera           8280
movie           7806
capella          119
Name: genre, dtype: int64

In [22]:
df['genre'] = genre_m1


In [23]:
df.set_index(["genre"],drop = False, inplace = True)


In [24]:
main_genres = ['rock','pop','blues','hip_hop','jazz','reggae','techno','trap','regga','rap','r&b']
def check_genre(sir):
    for word in main_genres:
        if sir.find(word) != -1:
            if word == 'rap':
                return 'hip_hop'
            else:
                return word
    return sir

df['genre'] = df['genre'].apply(check_genre)
genre = (df["genre"].str.strip()).str.lower()

In [25]:
genre_m1 = genre.copy()
genre_m1
genre_m1.value_counts()

reggae         17698
music          14756
comedy          9681
soundtrack      9646
indie           9543
jazz            9441
pop             9386
electronic      9377
folk            9299
hip-hop         9295
rock            9272
alternative     9263
classical       9256
hip_hop         9232
world           9096
soul            9089
blues           9023
r&b             8992
anime           8936
ska             8874
dance           8701
country         8664
opera           8280
movie           7806
capella          119
Name: genre, dtype: int64

reggae         17698
music          14756
comedy          9681
soundtrack      9646
indie           9543
jazz            9441
pop             9386
electronic      9377
folk            9299
hip-hop         9295
rock            9272
alternative     9263
classical       9256
hip_hop         9232
world           9096
soul            9089
blues           9023
r&b             8992
anime           8936
ska             8874
dance           8701
country         8664
opera           8280
movie           7806
capella          119
Name: genre, dtype: int64

In [26]:
unique = genre_m1.unique()


In [27]:
unique = genre_m1.unique()
to_remove = [] 

# genres that have a single instance only will be placed within the to_remove array
for genre in unique:
    if genre_m1.value_counts()[genre] < 200: # 10 was arbitrarily chosen
        to_remove += [genre]
len(to_remove)

1

1

In [28]:
df.set_index(["genre"],drop = False, inplace = True)
for name in to_remove:
    type(name)
    df.drop(index = str(name), inplace = True)


In [29]:
genre = (df["genre"].str.strip()).str.lower()


In [30]:
genre_m1 = genre.copy()
genre_m1
genre_m1.value_counts()

reggae         17698
music          14756
comedy          9681
soundtrack      9646
indie           9543
jazz            9441
pop             9386
electronic      9377
folk            9299
hip-hop         9295
rock            9272
alternative     9263
classical       9256
hip_hop         9232
world           9096
soul            9089
blues           9023
r&b             8992
anime           8936
ska             8874
dance           8701
country         8664
opera           8280
movie           7806
Name: genre, dtype: int64

reggae         17698
music          14756
comedy          9681
soundtrack      9646
indie           9543
jazz            9441
pop             9386
electronic      9377
folk            9299
hip-hop         9295
rock            9272
alternative     9263
classical       9256
hip_hop         9232
world           9096
soul            9089
blues           9023
r&b             8992
anime           8936
ska             8874
dance           8701
country         8664
opera           8280
movie           7806
Name: genre, dtype: int64

In [31]:
main_genres = ['rock','pop','blues','hip_hop','jazz','reggae','techno','trap','regga','rap','r&b']
def check_genre(sir):
    for word in main_genres:
        if sir.find(word) != -1:
            if word == 'rap':
                return 'hip hop'
            else:
                return word
    return sir

#df['genres'] = df['genres'].apply(check_genre)
#genre = (df["genres"].str.strip()).str.lower()


In [32]:
genre_m1 = genre.copy()
genre_m1
genre_m1.value_counts()

reggae         17698
music          14756
comedy          9681
soundtrack      9646
indie           9543
jazz            9441
pop             9386
electronic      9377
folk            9299
hip-hop         9295
rock            9272
alternative     9263
classical       9256
hip_hop         9232
world           9096
soul            9089
blues           9023
r&b             8992
anime           8936
ska             8874
dance           8701
country         8664
opera           8280
movie           7806
Name: genre, dtype: int64

reggae         17698
music          14756
comedy          9681
soundtrack      9646
indie           9543
jazz            9441
pop             9386
electronic      9377
folk            9299
hip-hop         9295
rock            9272
alternative     9263
classical       9256
hip_hop         9232
world           9096
soul            9089
blues           9023
r&b             8992
anime           8936
ska             8874
dance           8701
country         8664
opera           8280
movie           7806
Name: genre, dtype: int64

In [33]:
df


,genre,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
genre,,,,,,,,,,,,,,,
movie,movie,0,0.781665,0.389,99373,0.910,0.000000,C#,0.766953,-1.828,Major,0.691863,166.969,4/4,0.814
movie,movie,1,0.495984,0.590,137373,0.737,0.000000,F#,0.623368,-5.559,Minor,0.736741,174.003,4/4,0.816
movie,movie,3,0.975705,0.663,170267,0.131,0.000000,C,0.566512,-13.879,Minor,0.660448,99.488,5/4,0.368
movie,movie,0,0.838451,0.240,152427,0.326,0.000000,C#,0.560221,-12.178,Major,0.667690,171.758,4/4,0.227
movie,movie,4,0.974679,0.331,82625,0.225,0.769552,F,0.670406,-21.150,Major,0.679783,140.576,4/4,0.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
soul,soul,39,0.061968,0.687,326240,0.714,0.926723,D,0.539156,-10.626,Major,0.649323,115.542,4/4,0.962
soul,soul,38,0.181384,0.785,282447,0.683,0.415012,E,0.697730,-6.944,Minor,0.654566,113.830,4/4,0.969
soul,soul,47,0.949210,0.517,166960,0.419,0.000000,D,0.554444,-8.282,Major,0.787558,84.135,4/4,0.813


,genre,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
genre,,,,,,,,,,,,,,,
movie,movie,0,0.781665,0.389,99373,0.910,0.000000,C#,0.766953,-1.828,Major,0.691863,166.969,4/4,0.814
movie,movie,1,0.495984,0.590,137373,0.737,0.000000,F#,0.623368,-5.559,Minor,0.736741,174.003,4/4,0.816
movie,movie,3,0.975705,0.663,170267,0.131,0.000000,C,0.566512,-13.879,Minor,0.660448,99.488,5/4,0.368
movie,movie,0,0.838451,0.240,152427,0.326,0.000000,C#,0.560221,-12.178,Major,0.667690,171.758,4/4,0.227
movie,movie,4,0.974679,0.331,82625,0.225,0.769552,F,0.670406,-21.150,Major,0.679783,140.576,4/4,0.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
soul,soul,39,0.061968,0.687,326240,0.714,0.926723,D,0.539156,-10.626,Major,0.649323,115.542,4/4,0.962
soul,soul,38,0.181384,0.785,282447,0.683,0.415012,E,0.697730,-6.944,Minor,0.654566,113.830,4/4,0.969
soul,soul,47,0.949210,0.517,166960,0.419,0.000000,D,0.554444,-8.282,Major,0.787558,84.135,4/4,0.813


In [34]:
print(sns)

<module 'seaborn' from 'C:\\Users\\hemre\\Desktop\\odev\\env\\lib\\site-packages\\seaborn\\__init__.py'>
<module 'seaborn' from 'C:\\Users\\hemre\\Desktop\\odev\\env\\lib\\site-packages\\seaborn\\__init__.py'>


In [35]:
sns.catplot(data=df,x = 'genres', y='tempo', kind="bar")

ValueError: Could not interpret input 'genres'

ValueError: Could not interpret input 'genres'

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

In [ ]:
sns.distplot(df['tempo'])

In [ ]:
sns.pairplot(df[['tempo' ,'mode', 'energy']])

In [ ]:
sns.pairplot(df[['danceability' ,'loudness', 'liveness']])

In [ ]:
sns.pointplot(df[['danceability' ,'loudness', 'liveness']])

In [ ]:
#sns.catplot(data=df,x = 'energy', y='instrumentalness', kind="bar")

In [36]:
df

,genre,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
genre,,,,,,,,,,,,,,,
movie,movie,0,0.781665,0.389,99373,0.910,0.000000,C#,0.766953,-1.828,Major,0.691863,166.969,4/4,0.814
movie,movie,1,0.495984,0.590,137373,0.737,0.000000,F#,0.623368,-5.559,Minor,0.736741,174.003,4/4,0.816
movie,movie,3,0.975705,0.663,170267,0.131,0.000000,C,0.566512,-13.879,Minor,0.660448,99.488,5/4,0.368
movie,movie,0,0.838451,0.240,152427,0.326,0.000000,C#,0.560221,-12.178,Major,0.667690,171.758,4/4,0.227
movie,movie,4,0.974679,0.331,82625,0.225,0.769552,F,0.670406,-21.150,Major,0.679783,140.576,4/4,0.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
soul,soul,39,0.061968,0.687,326240,0.714,0.926723,D,0.539156,-10.626,Major,0.649323,115.542,4/4,0.962
soul,soul,38,0.181384,0.785,282447,0.683,0.415012,E,0.697730,-6.944,Minor,0.654566,113.830,4/4,0.969
soul,soul,47,0.949210,0.517,166960,0.419,0.000000,D,0.554444,-8.282,Major,0.787558,84.135,4/4,0.813


,genre,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
genre,,,,,,,,,,,,,,,
movie,movie,0,0.781665,0.389,99373,0.910,0.000000,C#,0.766953,-1.828,Major,0.691863,166.969,4/4,0.814
movie,movie,1,0.495984,0.590,137373,0.737,0.000000,F#,0.623368,-5.559,Minor,0.736741,174.003,4/4,0.816
movie,movie,3,0.975705,0.663,170267,0.131,0.000000,C,0.566512,-13.879,Minor,0.660448,99.488,5/4,0.368
movie,movie,0,0.838451,0.240,152427,0.326,0.000000,C#,0.560221,-12.178,Major,0.667690,171.758,4/4,0.227
movie,movie,4,0.974679,0.331,82625,0.225,0.769552,F,0.670406,-21.150,Major,0.679783,140.576,4/4,0.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
soul,soul,39,0.061968,0.687,326240,0.714,0.926723,D,0.539156,-10.626,Major,0.649323,115.542,4/4,0.962
soul,soul,38,0.181384,0.785,282447,0.683,0.415012,E,0.697730,-6.944,Minor,0.654566,113.830,4/4,0.969
soul,soul,47,0.949210,0.517,166960,0.419,0.000000,D,0.554444,-8.282,Major,0.787558,84.135,4/4,0.813


In [37]:
genre_m1.value_counts()

reggae         17698
music          14756
comedy          9681
soundtrack      9646
indie           9543
jazz            9441
pop             9386
electronic      9377
folk            9299
hip-hop         9295
rock            9272
alternative     9263
classical       9256
hip_hop         9232
world           9096
soul            9089
blues           9023
r&b             8992
anime           8936
ska             8874
dance           8701
country         8664
opera           8280
movie           7806
Name: genre, dtype: int64

reggae         17698
music          14756
comedy          9681
soundtrack      9646
indie           9543
jazz            9441
pop             9386
electronic      9377
folk            9299
hip-hop         9295
rock            9272
alternative     9263
classical       9256
hip_hop         9232
world           9096
soul            9089
blues           9023
r&b             8992
anime           8936
ska             8874
dance           8701
country         8664
opera           8280
movie           7806
Name: genre, dtype: int64

In [38]:
def transform(feature):
    le=LabelEncoder()
    df[feature]=le.fit_transform(df[feature])

In [39]:
cat_df=df.select_dtypes(include='object')
cat_df.columns

Index(['genre', 'key', 'mode', 'time_signature'], dtype='object')

Index(['genre', 'key', 'mode', 'time_signature'], dtype='object')

In [40]:
for col in cat_df.columns:
    transform(col)

df

,genre,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
genre,,,,,,,,,,,,,,,
movie,13,0,0.781665,0.389,99373,0.910,0.000000,4,0.766953,-1.828,0,0.691863,166.969,3,0.814
movie,13,1,0.495984,0.590,137373,0.737,0.000000,9,0.623368,-5.559,1,0.736741,174.003,3,0.816
movie,13,3,0.975705,0.663,170267,0.131,0.000000,3,0.566512,-13.879,1,0.660448,99.488,4,0.368
movie,13,0,0.838451,0.240,152427,0.326,0.000000,4,0.560221,-12.178,0,0.667690,171.758,3,0.227
movie,13,4,0.974679,0.331,82625,0.225,0.769552,8,0.670406,-21.150,0,0.679783,140.576,3,0.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
soul,21,39,0.061968,0.687,326240,0.714,0.926723,5,0.539156,-10.626,0,0.649323,115.542,3,0.962
soul,21,38,0.181384,0.785,282447,0.683,0.415012,7,0.697730,-6.944,1,0.654566,113.830,3,0.969
soul,21,47,0.949210,0.517,166960,0.419,0.000000,5,0.554444,-8.282,0,0.787558,84.135,3,0.813


,genre,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
genre,,,,,,,,,,,,,,,
movie,13,0,0.781665,0.389,99373,0.910,0.000000,4,0.766953,-1.828,0,0.691863,166.969,3,0.814
movie,13,1,0.495984,0.590,137373,0.737,0.000000,9,0.623368,-5.559,1,0.736741,174.003,3,0.816
movie,13,3,0.975705,0.663,170267,0.131,0.000000,3,0.566512,-13.879,1,0.660448,99.488,4,0.368
movie,13,0,0.838451,0.240,152427,0.326,0.000000,4,0.560221,-12.178,0,0.667690,171.758,3,0.227
movie,13,4,0.974679,0.331,82625,0.225,0.769552,8,0.670406,-21.150,0,0.679783,140.576,3,0.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
soul,21,39,0.061968,0.687,326240,0.714,0.926723,5,0.539156,-10.626,0,0.649323,115.542,3,0.962
soul,21,38,0.181384,0.785,282447,0.683,0.415012,7,0.697730,-6.944,1,0.654566,113.830,3,0.969
soul,21,47,0.949210,0.517,166960,0.419,0.000000,5,0.554444,-8.282,0,0.787558,84.135,3,0.813


In [41]:
#from bokeh.plotting import figure, show
#from bokeh.io import output_notebook

In [42]:
df.dtypes


genre                 int32
popularity            int64
acousticness        float64
danceability        float64
duration_ms           int64
energy              float64
instrumentalness    float64
key                   int32
liveness            float64
loudness            float64
mode                  int32
speechiness         float64
tempo               float64
time_signature        int32
valence             float64
dtype: object

genre                 int32
popularity            int64
acousticness        float64
danceability        float64
duration_ms           int64
energy              float64
instrumentalness    float64
key                   int32
liveness            float64
loudness            float64
mode                  int32
speechiness         float64
tempo               float64
time_signature        int32
valence             float64
dtype: object

In [43]:
X = df.drop('genre',axis=1)
Y = df.genre

In [44]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.15)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.15)

In [45]:
from sklearn.preprocessing import StandardScaler

standard_scaler = StandardScaler().fit(X_train)

# Standard Scaler
X_train_ST = standard_scaler.transform(X_train)
X_test_ST = standard_scaler.transform(X_test)

In [46]:
unique = np.unique(y_train)


In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

rand_forest = RandomForestClassifier(random_state = 42, min_samples_split = 5)
clf = LogisticRegression(multi_class = 'ovr')

In [48]:
df.dtypes


genre                 int32
popularity            int64
acousticness        float64
danceability        float64
duration_ms           int64
energy              float64
instrumentalness    float64
key                   int32
liveness            float64
loudness            float64
mode                  int32
speechiness         float64
tempo               float64
time_signature        int32
valence             float64
dtype: object

genre                 int32
popularity            int64
acousticness        float64
danceability        float64
duration_ms           int64
energy              float64
instrumentalness    float64
key                   int32
liveness            float64
loudness            float64
mode                  int32
speechiness         float64
tempo               float64
time_signature        int32
valence             float64
dtype: object

In [49]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

In [50]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
result_ST =[]
kfold = StratifiedKFold(n_splits = 5, random_state = 1, shuffle = True)
RF_cross_val_score = cross_val_score(rand_forest, X_train_ST, y_train, cv = kfold, scoring = 'accuracy')
print('%s: %f (%f)' % ('Random Forest', RF_cross_val_score.mean(), RF_cross_val_score.std()))

Random Forest: 0.393140 (0.001991)
Random Forest: 0.393140 (0.001991)


In [51]:
kfold = StratifiedKFold(n_splits = 5, random_state = 1, shuffle = True)
RF_cross_val_score = cross_val_score(clf, X_train_ST, y_train, cv = kfold, scoring = 'accuracy')
print('%s: %f (%f)' % ('Random Forest', RF_cross_val_score.mean(), RF_cross_val_score.std()))


KeyboardInterrupt


KeyboardInterrupt



In [ ]:
plt.figure(figsize=(20, 20))
sns.heatmap(df.corr(),annot=False)

In [ ]:
rs = np.random.RandomState(0)
df2 = pd.DataFrame(rs.rand(20,20))
corr = df2.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
from lazypredict.Supervised import LazyClassifier


In [ ]:
clf = LazyClassifier(verbose=-1,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

In [52]:
# build the lightgbm model
import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)

LGBMClassifier()

LGBMClassifier()

In [54]:
y_pred=clf.predict(X_test);

In [56]:
print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))

Test set score: 0.4397
Test set score: 0.4397


In [57]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.31      0.18      0.23      1394
           1       0.62      0.54      0.58      1352
           2       0.44      0.40      0.42      1310
           3       0.64      0.62      0.63      1423
           4       0.95      0.93      0.94      1500
           5       0.40      0.42      0.41      1301
           6       0.22      0.16      0.18      1325
           7       0.58      0.55      0.57      1404
           8       0.27      0.29      0.28      1441
           9       0.24      0.28      0.26      1442
          10       0.20      0.22      0.21      1362
          11       0.19      0.17      0.18      1457
          12       0.41      0.40      0.41      1384
          13       0.55      0.51      0.53      1159
          14       0.46      0.36      0.41      2172
          15       0.78      0.81      0.79      1283
          16       0.26      0.47      0.33      1360
          17       0.21    